In [9]:
import pandas as pd

In [10]:
BTC_15 = pd.read_csv("data/data/ohlcv/BTCUSDT_15m_20170208_20250830.csv")
ETH_15 = pd.read_csv("data/data/ohlcv/ETHUSDT_15m_20170208_20250830.csv")

In [23]:
df = BTC_15.copy()
df = df[df["timestamp"] >= "2025-05-03"]

In [24]:
# Calculate the split index
split_idx = int(len(df) * 1)  # first 60% train, last 40% test

# Split into train/test
train = df.iloc[:split_idx]
test  = df.iloc[split_idx:]

print("Train size:", len(train))
print("Test size:", len(test))

Train size: 11473
Test size: 0


In [25]:
train

,timestamp,open,high,low,close,volume
178998,2025-05-03 00:00:00+00:00,96833.5,96865.0,96744.0,96744.1,556.163
178999,2025-05-03 00:15:00+00:00,96744.1,96807.3,96677.9,96679.1,172.093
179000,2025-05-03 00:30:00+00:00,96679.1,96739.4,96600.2,96615.7,180.141
179001,2025-05-03 00:45:00+00:00,96615.7,96660.1,96569.7,96647.4,245.388
179002,2025-05-03 01:00:00+00:00,96647.4,96665.8,96506.6,96517.6,277.167
...,...,...,...,...,...,...
190466,2025-08-30 11:00:00+00:00,108404.7,108525.8,108400.0,108508.7,135.746
190467,2025-08-30 11:15:00+00:00,108508.7,108534.0,108436.6,108528.8,75.874
190468,2025-08-30 11:30:00+00:00,108528.8,108577.5,108486.9,108571.1,177.510
190469,2025-08-30 11:45:00+00:00,108571.1,108707.6,108571.1,108652.3,377.543


In [26]:
from Trading_algos.strategy_tester import StrategyTester

In [27]:
tester = StrategyTester(initial_capital=10000)

In [28]:
results = tester.run_backtest(
                            'FibonacciChannelStrategy',
                            train,
                            sensitivity=18,
                            leverage=1.0,
                            sl_percent=0,         # 1% adjustment to fib levels
                            fixed_stop=False,       # Use fib_786 (long) / fib_236 (short)
                            break_even_after_tp=1,  # Move to breakeven after TP2 hit
                            use_take_profits=True,
                            tp1_pct=1, tp1_close=40.0,
                            tp2_pct=2, tp2_close=30.0,
                            tp3_pct=3, tp3_close=20.0,
                            tp4_pct=4, tp4_close=10.0,
                            # Risk
                            risk_per_trade=0.015,   # 1.5% risk per trade
                            verbose=True
                        )

Starting backtest for FibonacciChannelStrategy
Data period: 2025-05-03 00:00:00+00:00 to 2025-08-30 12:00:00+00:00
Data points: 11473
Generating trading signals...
Running backtest simulation...


Backtesting: 100%|███████████████| 11473/11473 [00:07<00:00, 1635.35bars/s, Trades=119, Capital=$11,294, Position=FLAT]

Calculating performance metrics...

🎯 BACKTEST COMPLETED SUCCESSFULLY!
   Strategy: FibonacciChannelStrategy
   Total Trades: 119
   Final Return: 12.94%
   Final Capital: $11,293.67
   Win Rate: 62.18%
   Profit Factor: 2.41
   Max Drawdown: 3.87%
   Sharpe Ratio: 0.157
   Risk:Reward: 0.42
   Market Exposure: 1710.18%
   Total Fees: $599.24

💡 Use display_comprehensive_metrics('FibonacciChannelStrategy') for detailed analysis


In [9]:
# Cell 4: View detailed metrics
tester.display_comprehensive_metrics('FibonacciChannelStrategy')

# Cell 5: Plot results
#tester.plot_results('SimpleMAStrategy', results, show_plot=True)


📊 COMPREHENSIVE METRICS TABLE FOR FIBONACCICHANNELSTRATEGY

PERFORMANCE:
--------------------------------------------------
  Total Return        : -11.53%
  Final Capital       : $8,846.98
  Total Trades        : 186
  Win Rate            : 55.38%
  Profit Factor       : 1.01
  Risk:Reward         : 0.38

RISK METRICS:
--------------------------------------------------
  Max Drawdown        : 18.92%
  Sharpe Ratio        : -0.079
  Sortino Ratio       : -0.083
  Calmar Ratio        : -160.272
  Volatility          : 147.63%
  VaR (95%)           : -1.48%

TRADE ANALYSIS:
--------------------------------------------------
  Avg Win             : $38.91
  Avg Loss            : $102.14
  Largest Win         : $103.17
  Largest Loss        : $-152.96
  Max Consec Losses   : 5
  Max Consec Wins     : 17

STRATEGY BEHAVIOR:
--------------------------------------------------
  Trades/Day          : 1.60
  Avg Hold Time       : 22.39h
  Market Exposure     : 1623.27%
  Break-even          : 